<a href="https://colab.research.google.com/github/gdacanal/ElmerFEM_Colab/blob/main/FlowStep_ElmerFEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Open Google Colab (colab.research.google.com) and create a new Python 3 notebook.
#Install the required packages by running the following code in a code cell:
!add-apt-repository -y ppa:elmer-csc-ubuntu/elmer-csc-ppa
!apt-get update -qq
!apt-get install -y -qq elmerfem-csc

In [ ]:
# Upload the mesh file to Colab
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
import os

os.chdir('/content/drive/MyDrive/Colab Notebooks/ElmerColab/ElmerFEM_Colab')

In [ ]:
#List files in the current directory
%%shell
ls

In [ ]:
#ElmerGrid manual
#The first parameter defines the input file format:
#1) .grd : ElmerGrid file format
#2) .mesh.* : Elmer input format
#3) .ep : Elmer output format
#4) .ansys : Ansys input format
#5) .inp : Abaqus input format by Ideas
#6) .fil : Abaqus output format
#7) .FDNEUT : Gambit (Fidap) neutral file
#8) .unv : Universal mesh file format
#9) .mphtxt : Comsol Multiphysics mesh format
#10) .dat : Fieldview format
#11) .node,.ele: Triangle 2D mesh format
#12) .mesh : Medit mesh format
#13) .msh : GID mesh format
#14) .msh : Gmsh mesh format
#15) .ep.i : Partitioned ElmerPost format
#16) .2dm : 2D triangular FVCOM format

#The second parameter defines the output file format:
#1) .grd : ElmerGrid file format
#2) .mesh.* : ElmerSolver format (also partitioned .part format)
#3) .ep : ElmerPost format
#4) .msh : Gmsh mesh format
#5) .vtu : VTK ascii XML format

In [ ]:
#Convert mesh from "step.grd" to "step.msh" in ElmerGrid
%%shell
ElmerGrid 1 4 step.grd

In [ ]:
#Install meshio library
!pip install meshio

In [ ]:
import meshio
import matplotlib.pyplot as plt

mesh = meshio.read('step.msh')

# Extract the mesh points and cells information
points = mesh.points
cells = mesh.cells[0]  # Assuming there is only one cell type in the MSH file

# Plot the mesh using Matplotlib
fig, ax = plt.subplots()

# Plot the mesh cells
if cells.type == "quad":
    cell_quads = cells.data
    for quad in cell_quads:
        quad_points = points[quad]
        quad_points = [*quad_points, quad_points[0]]  # Close the quad
        xs, ys, _ = zip(*quad_points)
        ax.plot(xs, ys, 'b-')
elif cells.type == "triangle":
    cell_triangles = cells.data
    for triangle in cell_triangles:
        triangle_points = points[triangle]
        triangle_points = [*triangle_points, triangle_points[0]]  # Close the triangle
        xs, ys, _ = zip(*triangle_points)
        ax.plot(xs, ys, 'b-')
elif cells.type == "line":
    cell_lines = cells.data
    for line in cell_lines:
        line_points = points[line]
        xs, ys, _ = zip(*line_points)
        ax.plot(xs, ys, 'b-')

# Set the aspect ratio to 'equal'
ax.set_aspect('equal')

# Display the plot
plt.show()

In [ ]:
#Convert mesh from "step.grd" to "step" ElmerFEM solver format
%%shell
ElmerGrid 1 2 step.grd

In [ ]:
#List files in the current directory

%%shell
ls

In [ ]:
# Create Elmer sif file
sif_content = '''
Header
  CHECK KEYWORDS Warn
  Mesh DB "." "step"
  Include Path ""
  Results Directory ""
End

Simulation
  Max Output Level = 4
  Coordinate System = Cartesian
  Coordinate Mapping(3) = 1 2 3
  Simulation Type = Steady state
  Steady State Max Iterations = 1
  Output Intervals(1) = 1
  Solver Input File = case.sif
  Post File = case.vtu
End

Constants
  Gravity(4) = 0 -1 0 9.82
  Stefan Boltzmann = 5.67e-08
  Permittivity of Vacuum = 8.8542e-12
  Permeability of Vacuum = 1.25663706e-6
  Boltzmann Constant = 1.3807e-23
  Unit Charge = 1.602e-19
End

Body 1
  Target Bodies(1) = 1
  Name = "Body Property 1"
  Equation = 1
  Material = 1
End

Solver 1
  Equation = Navier-Stokes
  Procedure = "FlowSolve" "FlowSolver"
  Variable = Flow Solution[Velocity:2 Pressure:1]
  Exec Solver = Always
  Stabilize = True
  Optimize Bandwidth = True
  Steady State Convergence Tolerance = 1.0e-5
  Nonlinear System Convergence Tolerance = 1.0e-8
  Nonlinear System Max Iterations = 20
  Nonlinear System Newton After Iterations = 3
  Nonlinear System Newton After Tolerance = 1.0e-3
  Nonlinear System Relaxation Factor = 1
  Linear System Solver = Iterative
  Linear System Iterative Method = BiCGStab
  Linear System Max Iterations = 500
  Linear System Convergence Tolerance = 1.0e-8
  BiCGstabl polynomial degree = 2
  Linear System Preconditioning = ILU0
  Linear System ILUT Tolerance = 1.0e-3
  Linear System Abort Not Converged = False
  Linear System Residual Output = 1
  Linear System Precondition Recompute = 1
End

Equation 1
  Name = "Navier-Stokes"
  Active Solvers(1) = 1
End

Material 1
  Name = "Ideal"
  Density = 1.0
  Viscosity = 0.01
End

Boundary Condition 1
  Target Boundaries(1) = 1 
  Name = "Inlet"
  Velocity 2 = 0.0
  Velocity 1 = Variable Coordinate 2; Real MATC "6*(tx-1)*(2-tx)"
End

Boundary Condition 2
  Target Boundaries(1) = 2 
  Name = "Outlet"
  Velocity 2 = 0.0
End

Boundary Condition 3
  Target Boundaries(1) = 3 
  Name = "Walls"
  Noslip wall BC = True
End
'''

In [ ]:
# Save sif file
with open('case.sif', 'w') as f:
    f.write(sif_content)

In [ ]:
%%shell
ElmerSolver case.sif

In [ ]:
#List files in the current directory

%%shell
ls
cd step
ls

In [ ]:
import meshio
import numpy as np
import matplotlib.pyplot as plt

# Read VTU file
mesh = meshio.read("./step/case_t0001.vtu")

# Extract the points and cells from the mesh
points = mesh.points
cells = mesh.cells

# Extract velocity data
velocity = mesh.point_data['velocity']

# Plotting 2D mesh with velocity data
fig, ax = plt.subplots()

# Plotting vertices with velocity data
ax.quiver(points[:, 0], points[:, 1], velocity[:, 0], velocity[:, 1])

# Plotting cells (triangles or tetrahedra)
if 'triangle' in cells:
    triangles = cells['triangle']
    ax.tripcolor(points[:, 0], points[:, 1], triangles=triangles, facecolors='gray', alpha=0.5)
elif 'tetra' in cells:
    tetrahedra = cells['tetra']
    ax.tripcolor(points[:, 0], points[:, 1], triangles=tetrahedra, facecolors='gray', alpha=0.5)

# Set aspect ratio
ax.set_aspect(1)  # or any other desired aspect ratio

# Set axis labels
ax.set_xlabel('X')
ax.set_ylabel('Y')

# Show the plot
plt.show()